In [1]:
import re
import math
import keras
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

2025-08-25 06:28:01.245011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-25 06:28:01.276044: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-25 06:28:01.286244: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-25 06:28:01.395744: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    details = tf.config.experimental.get_device_details(gpu)
    compute_capability = details.get('compute_capability', 'Unknown')
    print(f"Device: {gpu.name}, Compute Capability: {compute_capability}")
    
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Device: /physical_device:GPU:0, Compute Capability: (7, 5)


I0000 00:00:1756103285.859574    7105 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1756103286.107716    7105 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1756103286.108110    7105 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1756103286.110084    7105 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
MODEL_PATH = "../model/"
MODEL_PATH += "softplus_cnn_7/"

In [4]:
def print_mnist(ds):
    import matplotlib.pyplot as plt

    # Function to display MNIST images
    def show_images(images, labels):
        plt.figure(figsize=(10, 5))
        for i in range(10):
            plt.subplot(2, 5, i + 1)
            plt.imshow(images[i].numpy(), cmap='gray')
            plt.title(f"Label: {labels[i].numpy()}")
            plt.axis("off")
        plt.show()

    # Display a few images from the training set
    num_images_to_display = 10
    train_images, train_labels = next(iter(ds.batch(num_images_to_display)))
    show_images(train_images, train_labels)


In [5]:
(_ds_train, _ds_test), _ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

I0000 00:00:1756103286.187498    7105 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1756103286.187869    7105 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1756103286.188096    7105 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1756103286.307406    7105 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [6]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

In [7]:
ds_train = _ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(_ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [8]:
ds_test = _ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [9]:
model = keras.models.load_model(f'{MODEL_PATH}model.keras')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 12, 12, 5)      │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 12, 12, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 6, 6, 5)        │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 180)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,822 (22.75 KB)

 Trainable params: 1,940 (7.58 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,882 (15.17 KB)

In [10]:
for t_v in model.variables: # changed from trainable_variables
    print("name of weight: ", t_v.name)
    print("shape of the weight", t_v.shape)
    print("value type: ", t_v.numpy().dtype)
    print()

name of weight:  kernel
shape of the weight (5, 5, 1, 5)
value type:  float32

name of weight:  bias
shape of the weight (5,)
value type:  float32

name of weight:  kernel
shape of the weight (180, 10)
value type:  float32

name of weight:  bias
shape of the weight (10,)
value type:  float32



In [11]:
model.trainable_variables[2].path

'dense/kernel'

In [12]:
for t_v in model.variables:
    _name = t_v.path
    
    if "_" not in _name.split("/")[0] or _name.split("/")[0] == "batch_normalization":
        name = _name.split("/")[0] + "_0/" + _name.split("/")[1]
    else:
        name = _name
    
    name = name.replace("/", "_")
    
    if _name.split("/")[1] == "gamma":
        continue
    elif _name.split("/")[1] == "moving_variance":
        target_path = t_v.path
        for layer in model.layers:
            for var in layer.variables:
                if var.path == target_path:
                    print("BN epsilon:", layer.epsilon)
                    
                    moving_variance = layer.moving_variance.numpy()
                    epsilon = layer.epsilon
                    gamma = layer.gamma.numpy()

                    bn_denominator = np.sqrt(moving_variance + epsilon)
                    print(bn_denominator)
                    bn_denominator = gamma/bn_denominator
                    print(bn_denominator)
                    np.save(f'{MODEL_PATH}{name}_gamma', bn_denominator.flatten())
                    print(f'{_name.split("/")[1]} type: {bn_denominator.dtype}')
                    print(f'{_name.split("/")[1]} file: {name}')
                    break
    elif _name.split("/")[1] == "moving_mean":
        moving_mean = t_v.numpy()
        print(moving_mean)
        moving_mean = -moving_mean
        print(moving_mean)
        np.save(f'{MODEL_PATH}{name}', moving_mean.flatten())
        print(f'{_name.split("/")[1]} type: {moving_mean.dtype}')
        print(f'{_name.split("/")[1]} file: {name}')
    else:
        np.save(f'{MODEL_PATH}{name}', t_v.numpy().flatten())
        print(f'{_name.split("/")[1]} type: {t_v.numpy().dtype}')
        print(f'{_name.split("/")[1]} file: {name}')

kernel type: float32
kernel file: conv2d_0_kernel
bias type: float32
bias file: conv2d_0_bias
kernel type: float32
kernel file: dense_0_kernel
bias type: float32
bias file: dense_0_bias


In [13]:
x_ls = []
y_ls = []
for x, y in iter(ds_test.unbatch()):
    x_ls.append(x)
    y_ls.append(y)

x_np = np.array(x_ls)
y_np = np.array(y_ls)

select_el = 500
print(x_np[select_el].shape, y_ls[select_el])

np.save(f"{MODEL_PATH}test_x", x_np.flatten()) # .map(lambda x: tf.cast(x, tf.float32))
np.save(f"{MODEL_PATH}test_y", y_np.flatten())

print(x_np.dtype)
print(y_np.dtype)

2025-08-25 06:28:10.036888: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


(28, 28, 1) tf.Tensor(2, shape=(), dtype=int64)
float32
int64


In [14]:
poly_str = "+0.8911666682660481*x**0+0.4926144253738015*x**1+0.06258069171632989*x**2+0.0004330747691722465*x**3-0.00023733607901459163*x**4-4.411371285219169e-06*x**5+3.9236230531557943e-07*x**6+1.0966835822404214e-08*x**7"
domain = None
level = 0

terms = re.findall(r'[+-]?\d*\.?\d+(?:e[+-]?\d+)?', poly_str)
consts = [float(term) for term in terms]
poly_consts = np.array([v for i, v in enumerate(consts) if i % 2 != 1]) #, dtype=np.float32)

np.save(f"{MODEL_PATH}poly_consts_{level}", poly_consts.flatten())

if domain:
    poly_domain = np.array([1/domain, domain])
    np.save(f"{MODEL_PATH}poly_domain_{level}", poly_domain.flatten())

In [15]:
poly_consts

array([ 8.91166668e-01,  4.92614425e-01,  6.25806917e-02,  4.33074769e-04,
       -2.37336079e-04, -4.41137129e-06,  3.92362305e-07,  1.09668358e-08])

In [16]:
poly_consts.dtype

dtype('float64')

In [17]:
if domain:
    poly_domain

In [18]:
if domain:
    poly_domain.dtype